In [1]:
from docling.document_converter import DocumentConverter

/opt/miniconda3/envs/azureai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [82]:
file_path = 'data/apple_FY24_report.pdf'

converter = DocumentConverter()

doc = converter.convert(str(file_path))

markdown_content = doc.document.export_to_markdown()

In [83]:
markdown_content

"## UNITED STATES SECURITIES AND EXCHANGE COMMISSION\n\nWashington, D.C. 20549\n\n## FORM 10-K\n\n<!-- image -->\n\n(Mark One)\n\n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the fiscal year ended September 28, 2024\n\nor\n\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\n\nFor the transition period from              to             .\n\nCommission File Number: 001-36743\n\n## Apple Inc.\n\n(Exact name of Registrant as specified in its charter)\n\nCalifornia\n\n94-2404110\n\n(State or other jurisdiction of incorporation or organization)\n\n(I.R.S. Employer Identification No.)\n\nOne Apple Park Way Cupertino, California\n\n95014\n\n(Address of principal executive offices)\n\n(Zip Code)\n\n## (408) 996-1010\n\n(Registrant's telephone number, including area code)\n\nSecurities registered pursuant to Section 12(b) of the Act:\n\nTitle of each class\n\nTrading symbol(s)\n\nName of each exchange on wh

In [9]:
from langchain.text_splitter import MarkdownHeaderTextSplitter


In [84]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
raw_chunks = splitter.split_text(doc.document.export_to_markdown())

# Add metadata to each chunk
chunks = []
for chunk in raw_chunks:
    chunk.metadata.update({
        "source": file_path,
        "content_type": "text"
    })
    chunks.append(chunk)

In [64]:
chunks

[Document(metadata={'Header 2': 'UNITED STATES SECURITIES AND EXCHANGE COMMISSION', 'source': 'data/apple_FY24_report.pdf', 'content_type': 'text'}, page_content='Washington, D.C. 20549'),
 Document(metadata={'Header 2': 'FORM 10-K', 'source': 'data/apple_FY24_report.pdf', 'content_type': 'text'}, page_content='<!-- image -->  \n(Mark One)  \n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934  \nFor the fiscal year ended September 28, 2024  \nor  \n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934  \nFor the transition period from              to             .  \nCommission File Number: 001-36743'),
 Document(metadata={'Header 2': 'Apple Inc.', 'source': 'data/apple_FY24_report.pdf', 'content_type': 'text'}, page_content='(Exact name of Registrant as specified in its charter)  \nCalifornia  \n94-2404110  \n(State or other jurisdiction of incorporation or organization)  \n(I.R.S. Employer Identification No.)  \

In [65]:
len(chunks)

251

In [85]:
file_path_2 = 'data/MSFT_FY24Q4_10K.pdf'

converter2 = DocumentConverter()

doc2 = converter2.convert(str(file_path_2))

markdown_content2 = doc2.document.export_to_markdown()

In [86]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
raw_chunks2 = splitter.split_text(doc2.document.export_to_markdown())

# Add metadata to each chunk
for chunk in raw_chunks2:
    chunk.metadata.update({
        "source": file_path_2,
        "content_type": "text"
    })
    chunks.append(chunk)

In [68]:
 chunks

[Document(metadata={'Header 2': 'UNITED STATES SECURITIES AND EXCHANGE COMMISSION', 'source': 'data/apple_FY24_report.pdf', 'content_type': 'text'}, page_content='Washington, D.C. 20549'),
 Document(metadata={'Header 2': 'FORM 10-K', 'source': 'data/apple_FY24_report.pdf', 'content_type': 'text'}, page_content='<!-- image -->  \n(Mark One)  \n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934  \nFor the fiscal year ended September 28, 2024  \nor  \n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934  \nFor the transition period from              to             .  \nCommission File Number: 001-36743'),
 Document(metadata={'Header 2': 'Apple Inc.', 'source': 'data/apple_FY24_report.pdf', 'content_type': 'text'}, page_content='(Exact name of Registrant as specified in its charter)  \nCalifornia  \n94-2404110  \n(State or other jurisdiction of incorporation or organization)  \n(I.R.S. Employer Identification No.)  \

In [95]:
len(chunks)

598

In [99]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import os

persist_dir = "./chroma_db3"

settings = chromadb.Settings(
            is_persistent=True,
            persist_directory=persist_dir,
            anonymized_telemetry=False
        )

embeddings = OpenAIEmbeddings(
            model=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
        )


vector_store = Chroma(
            collection_name="financial_docs",
            embedding_function=embeddings,
            persist_directory=persist_dir
        )   

In [97]:
chunks

[Document(metadata={'Header 2': 'UNITED STATES SECURITIES AND EXCHANGE COMMISSION', 'source': 'data/apple_FY24_report.pdf', 'content_type': 'text'}, page_content='Washington, D.C. 20549'),
 Document(metadata={'Header 2': 'FORM 10-K', 'source': 'data/apple_FY24_report.pdf', 'content_type': 'text'}, page_content='<!-- image -->  \n(Mark One)  \n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934  \nFor the fiscal year ended September 28, 2024  \nor  \n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934  \nFor the transition period from              to             .  \nCommission File Number: 001-36743'),
 Document(metadata={'Header 2': 'Apple Inc.', 'source': 'data/apple_FY24_report.pdf', 'content_type': 'text'}, page_content='(Exact name of Registrant as specified in its charter)  \nCalifornia  \n94-2404110  \n(State or other jurisdiction of incorporation or organization)  \n(I.R.S. Employer Identification No.)  \

In [100]:
batch_size = 5000

for i in range(0, len(chunks), batch_size):
    vector_store.add_documents(chunks[i:i + batch_size])

In [112]:
prompt = "What is the R&D cost of Apple from 2022 to 2024 as compared to microsoft?"

In [113]:
from openai import AzureOpenAI

client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

In [114]:
def expand_query_with_llm(query):
    """Use LLM to generate query expansion terms."""
    expansion_prompt = f"""
    Expand the following financial query by providing synonyms, related concepts, and alternative phrasings. 
    Ensure the terms remain relevant to financial data analysis:
    
    Query: "{query}"
    
    Provide the expansion terms as a comma-separated list.
    """
    
    response = client.chat.completions.create(
        model="gpt-4o",  # Use GPT-4 or a suitable model
        messages=[{"role": "user", "content": expansion_prompt}],
        max_tokens=50,
        temperature=0.5  # Lower temp for more deterministic outputs
    )
    
    # Extract expansion terms
    expansion_terms = response.choices[0].message.content.strip().split(", ")
    
    # Combine with the original query
    expanded_query = query + " " + " ".join(expansion_terms)
    
    return expanded_query

In [115]:
expand_query_with_llm(prompt)

'What is the R&D cost of Apple from 2022 to 2024 as compared to microsoft? "research and development expenditure of Apple from 2022 to 2024 versus Microsoft R&D expenses of Apple between 2022 and 2024 compared to Microsoft Apple\'s R&D spending from 2022 to 2024 in comparison'

In [116]:
relevant_docs = vector_store.similarity_search(expand_query_with_llm(prompt), k=12)

context = "\n".join([doc.page_content for doc in relevant_docs])

messages = [
    {"role": "system", "content": f"""
    You are a financial data analyst assistant that helps analyze financial data from PDFs.
    
    Use this context from the documents to answer the question {prompt}:
    {context}
    
    """}
]

response = client.chat.completions.create(
    model=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    messages=messages,
    temperature=0
)

response_content = response.choices[0].message.content

response_content

"From 2022 to 2024, the R&D costs for Apple and Microsoft are as follows:\n\n**Apple:**\n- 2024: $31,370 million\n- 2023: $29,915 million\n- 2022: $26,251 million\n\n**Microsoft:**\n- 2024: $29,510 million\n- 2023: $27,195 million\n- 2022: $24,512 million\n\nComparing the R&D costs:\n- In 2024, Apple's R&D cost was $31,370 million, while Microsoft's was $29,510 million.\n- In 2023, Apple's R&D cost was $29,915 million, while Microsoft's was $27,195 million.\n- In 2022, Apple's R&D cost was $26,251 million, while Microsoft's was $24,512 million.\n\nOverall, Apple consistently spent more on R&D than Microsoft each year from 2022 to 2024."

In [117]:
relevant_docs

[Document(metadata={'Header 2': 'Research and Development', 'content_type': 'text', 'source': 'data/MSFT_FY24Q4_10K.pdf'}, page_content='| (In millions, except percentages)   | 2024     | 2023     | Percentage  Change   |\n|-------------------------------------|----------|----------|----------------------|\n| Research and development            | $ 29,510 | $ 27,195 | 9%                   |\n| As a percent of revenue             | 12%      | 13%      | (1)ppt               |  \nResearch and development expenses include payroll, employee benefits, stock-based compensation expense, and other headcount-related expenses associated with product development. Research and development expenses also include third-party development and programming costs and the amortization of purchased software code and services content.'),
 Document(metadata={'Header 2': 'Operating Expenses', 'content_type': 'text', 'source': 'data/apple_FY24_report.pdf'}, page_content='Operating expenses for 2024, 2023 and 20

In [111]:
relevant_docs[0].page_content

'The following table shows net sales by category for 2024, 2023 and 2022 (dollars in millions):  \n|                                 | 2024      | Change   | 2023      | Change   | 2022      |\n|---------------------------------|-----------|----------|-----------|----------|-----------|\n| iPhone                          | $ 201,183 | - %      | $ 200,583 | (2)%     | $ 205,489 |\n| Mac                             | 29,984    | 2 %      | 29,357    | (27)%    | 40,177    |\n| iPad                            | 26,694    | (6)%     | 28,300    | (3)%     | 29,292    |\n| Wearables, Home and Accessories | 37,005    | (7)%     | 39,845    | (3)%     | 41,241    |\n| Services  (1)                   | 96,169    | 13 %     | 85,200    | 9 %      | 78,129    |\n| Total net sales                 | $ 391,035 | 2 %      | $ 383,285 | (3)%     | $ 394,328 |'